In [1]:
#!pip install ipynb

In [1]:
import pygame as pg
from random import sample

from ipynb.fs.full.nlp import *

pygame 2.1.2 (SDL 2.0.18, Python 3.9.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
[remove_unseen_words] OUTPUT: pdas not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: zus not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: anaheim not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: mpegs not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: greensboro not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: usps not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: mrna not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: cdna not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: liechtenstein not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: swaziland not found in fasttext -> excluded from words list
[remove_unseen

In [2]:
# similarity_matrix = pd.read_csv('../data/similarity_matrix.csv', index_col=0).drop_duplicates()
# print(similarity_matrix.shape)

# hint_words = load_hint_words()
# game_words = load_codename_words()

# board = get_example_board(game_words)
# print(board)

# print("Giving hint...")
# hint = hint_giver(similarity_matrix, unknown_words)
# print(hint) 

In [14]:
switch={
    pg.K_ESCAPE:'E',       #Exit the program
    pg.K_r:'R',                 #Used to restart game
    pg.K_s:'S',                 #Used to end turn
    'MOUSE_PRESS':'MOUSE_PRESS' #Used to select a cell and indicate that an action should be performed
}       
        
def keypress(key):
    """ Given a pressed key, check if we care whether this key has been pressed. """
    return switch.get(key, "SP_NONE")

def getKey():
    """ Looks for the states of a set of keys and whether the QUIT key has been pressed. """
    key = None
    for event in pg.event.get():                   #For all pygame events
        if event.type == pg.QUIT:                  #If the user is trying to quit
            key = pg.K_ESCAPE                      #Handle as an escape key
        elif event.type == pg.KEYDOWN:             #If the user is pressing a key
            key = event.key                        #Get the key
        elif event.type == pg.MOUSEBUTTONDOWN:     #If the user has used a mouse button
            mouse_presses = pg.mouse.get_pressed() #Get what they've pressed
            if mouse_presses[0]:                   #If they've pressed left mouse
                key = 'MOUSE_PRESS'                #Set the key as a mousepress
    return keypress(key)                           #Return the result of the switch statement

In [29]:
class UserInterface:
    def __init__(self):
        #Load all images
        self.cell_r_img = pg.image.load("../img/cellr.png")
        self.cell_b_img = pg.image.load("../img/cellb.png")
        self.cell_x_img = pg.image.load("../img/cellx.png")
        self.cell_n_img = pg.image.load("../img/celln.png")
        self.cell_u_img = pg.image.load("../img/cellu.png")
        self.bg_img     = pg.image.load("../img/bg.png")
        
        #Set the size of every cell and the size of the font
        self.cell_size = (200, 100)
        self.font_size = 24
        self.font = pg.font.SysFont("calibri", self.font_size)
        
        #Set the number of cells to display on the x and y axes
        self.cells_x = 7
        self.cells_y = 6
        
        #Set which cell the board should begin on
        self.chest_start_pos = 1
        self.chest_endX_pos = 5
        
        #Initialize the game log
        self.game_log = []
        
        #Create the screen
        self.screen = pg.display.set_mode((self.cells_x*self.cell_size[0], self.cells_y*self.cell_size[1]))
    def update_screen(self, CellStates, TurnTracker, clue):
        #Update every component of the game board
        self.update_background()
        self.update_cells(CellStates, self.cell_size[1]*self.chest_start_pos)
        self.update_score(CellStates)
        self.update_turns(TurnTracker)
        self.update_clue(TurnTracker, clue)
        self.update_game_log()
        pg.display.update()
    def update_cells(self, CellStates, y_offset=0):
        r_img = pg.transform.scale(self.cell_r_img, self.cell_size)
        b_img = pg.transform.scale(self.cell_b_img, self.cell_size)
        x_img = pg.transform.scale(self.cell_x_img, self.cell_size)
        n_img = pg.transform.scale(self.cell_n_img, self.cell_size)
        u_img = pg.transform.scale(self.cell_u_img, self.cell_size)
        for x in range(0, 5):
            for y in range(0, 5):
                cell_state = CellStates.get_cell_state((x, y))
                pos = (self.cell_size[0]*x, self.cell_size[1]*y + y_offset)
                word_type = cell_state.word_type
                known = cell_state.known
                
                #Display cell card
                if word_type == "R" and known:
                    self.screen.blit(r_img, pos)
                elif word_type == "B" and known:
                    self.screen.blit(b_img, pos)
                elif word_type == "X" and known:
                    self.screen.blit(x_img, pos)
                elif word_type == "N" and known:
                    self.screen.blit(n_img, pos)
                else:
                    self.screen.blit(u_img, pos)
                
                #Display cell word
                word = self.font.render(cell_state.word, 1, (0,0,0))                
                self.screen.blit(word, (pos[0] + self.cell_size[0]//3, pos[1] + self.cell_size[1]//3))
    def update_score(self, CellStates):
        num_red, num_blu = CellStates.get_num_unknown_rb_words()
        #Display cell word
        red_rem = self.font.render("Red words remaining: " + str(num_red), 1, (255,100,100))
        blu_rem = self.font.render("Blue words remaining: " + str(num_blu), 1, (100,100,255))
        self.screen.blit(red_rem, (self.cell_size[0]//3, self.cell_size[1]//3))
        self.screen.blit(blu_rem, ((self.cells_x)*self.cell_size[0] - self.cell_size[0]//3*5, self.cell_size[1]//3))
    def update_turns(self, TurnTracker):
        if TurnTracker.is_game_ongoing():
            msg = "Waiting for "
            msg += "red" if TurnTracker.is_red_turn() else "blue"
            msg += " to "
            msg += "give a clue." if TurnTracker.is_codemaster_turn() else "make a guess."
            color = (255, 100, 100) if TurnTracker.is_red_turn() else (100, 100, 255)
        else:
            msg = self.game_log[-1]
            color = (100, 255, 100)

        message = self.font.render(msg, 1, color)
        self.screen.blit(message, ((self.cells_x)*(self.cell_size[0]//3), self.cell_size[1]//3))
    def update_clue(self, TurnTracker, clue):
        #If there's a clue on the board
        if clue:
            msg = clue[0] + ", " + str(clue[1])
            message = self.font.render(msg, 1, (255,255,255))
            self.screen.blit(message, ((self.cells_x)*(self.cell_size[0]//3), 2*self.cell_size[1]//3))
    def update_background(self):
        bg_img = pg.transform.scale(self.bg_img, self.cell_size)
        for x in range(0, self.cells_x):
            for y in range(0, self.cells_y):
                pos = (self.cell_size[0]*x, self.cell_size[1]*y)
                #Display cell background
                self.screen.blit(bg_img, pos)
    
    def console_print(self, message):
        self.game_log.append(message)

    def update_game_log(self):
        # Shows the last 20 messages
        log_show = []
        length_log = len(self.game_log)
        pos_y = self.chest_start_pos * self.cell_size[1]
        for x in range(len(self.game_log)):
            if length_log > 19:
                length_log -= 1
                continue
            log_show.append(self.game_log[x])

        for m in log_show:
            message = self.font.render(m, False, (255, 255, 255))
            self.screen.blit(message, (self.chest_endX_pos*self.cell_size[0], pos_y))
            pos_y += (self.font_size + 2)

    def get_mouse_cell_pos(self, mouse_real_pos):
        """ Returns the cell that has been clicked in the form of an integer """
        return (mouse_real_pos[0]//self.cell_size[0], mouse_real_pos[1]//self.cell_size[1])
                     
    def __del__(self):
        pg.display.quit()
        

In [30]:
class CellState:
    def __init__(self, word="NOT SET", word_type="N"):
        self.word = word
        self.word_type = word_type
        self.known = False

class CellStateHandler:
    def __init__(self, wordlist, grid_size):
        #Convert all words in wordlist to lowercase
        wordlist = [w.lower() for w in wordlist]
        
        #Generate word cards by randomly selecting words from the list, then choose their state
        selected_words = sample(wordlist, k=25)
        assigned_words = sample(selected_words, k=25)
        red_words = assigned_words[0:9]
        blue_words = assigned_words[9:17]
        fail_words = [assigned_words[17]]
        
        self.x_size, self.y_size = grid_size
        
        self.cell_states = []
        self.red_unk, self.blu_unk, self.neu_unk, self.fai_unk = ([],[],[],[])
        for x in range(0, self.x_size):
            cell_state_arr = []
            for y in range(0, self.y_size):
                index_1D = x + y*self.x_size
                #Set the cell state (Red, Blue, Fail, or Neutral)
                if selected_words[index_1D] in red_words:
                    word_type = "R"
                    self.red_unk.append(selected_words[index_1D])
                elif selected_words[index_1D] in blue_words:
                    word_type = "B"
                    self.blu_unk.append(selected_words[index_1D])
                elif selected_words[index_1D] in fail_words:
                    word_type = "X"
                    self.fai_unk.append(selected_words[index_1D])
                else:
                    word_type = "N"
                    self.neu_unk.append(selected_words[index_1D])
                cell_state_arr.append(CellState(selected_words[index_1D], word_type))
                
            self.cell_states.append(cell_state_arr)
    def get_cell_state(self, pos):
        """ Returns all information pertaining to the current cell """
        return self.cell_states[pos[0]][pos[1]]
    def set_flag_known(self, pos):
        """ Sets the known flag of the given cell as true. Returns true if the state of the cell was already known. """
        #If the word isn't already set to known
        if not self.cell_states[pos[0]][pos[1]].known:
            #Set the word to known
            self.cell_states[pos[0]][pos[1]].known = True
            
            #Remove the word from the corresponding unknown word list
            word, word_type = (self.cell_states[pos[0]][pos[1]].word, self.cell_states[pos[0]][pos[1]].word_type)
            if word_type == "R":
                self.red_unk.remove(word)
            elif word_type == "B":
                self.blu_unk.remove(word)
            elif word_type == "X":
                self.fai_unk.remove(word)
            elif word_type == "N":
                self.neu_unk.remove(word)
        return word, word_type
    def get_flag_known(self, pos):
        return self.cell_states[pos[0]][pos[1]].known
    def get_num_unknown_rb_words(self):
        return len(self.red_unk), len(self.blu_unk)
    def get_all_unknown_words(self):
        return (self.red_unk, self.blu_unk, self.neu_unk, self.fai_unk)

In [31]:
def get_wordlist():
    wordlist = []
    with open("../data/codename_words.txt", 'r') as wordfile:
        lines = wordfile.readlines()
        for line in lines:
            wordlist.append(line[0:-1])
    return wordlist

In [32]:
class turn_tracker:
    """ Turn counter for entire game
        turn % 2 == 0 - It is a AI's turn
        turn % 2 == 1 - It is a player's turn
        turn % 4 in [0,1] - It is red's turn
        turn % 4 in [2,3] - It is blue's turn
    """
    def __init__(self):
        self.turn = 0
        self.game_finished = False
    def get_turn(self):
        return self.turn
    def increment_turn(self):
        self.turn += 1
    def is_red_turn(self):
        return self.turn % 4 in [0, 1]
    def is_blue_turn(self):
        return not self.is_red_turn()
    def is_codemaster_turn(self):
        return self.turn % 2 == 0
    def is_guesser_turn(self):
        return not self.is_codemaster_turn()
    def is_game_finished(self):
        return self.game_finished
    def is_game_ongoing(self):
        return not self.is_game_finished()
    def set_game_finished(self):
        self.game_finished = True

In [34]:
def main(UI):
    
    continuing = True
    while continuing:
        UI.console_print("A new game was started")
        
        #Get cell state information based on the wordlist
        CellStates = CellStateHandler(load_codename_words(), (5,5))
        unknown_words = CellStates.get_all_unknown_words()
        
        CCG = CodenamesClueGenerator(unknown_words)

        #Set the user's selected key as none
        key = "SP_INITIAL"
        mouse_real_pos = pg.mouse.get_pos() #Get the current location of the mouse

        TurnTracker = turn_tracker()

        clue = None

        #While the user isn't trying to quit
        done = False
        while not done:
            UI.update_screen(CellStates, TurnTracker, clue)

            key = getKey() #Get any relevant keys being pressed
            mouse_real_pos = pg.mouse.get_pos() #Get the current location of the mouse

            """ AI Operations """
            if TurnTracker.is_codemaster_turn():

                if TurnTracker.is_red_turn():
                    team_words, enemy_words, neutral_words, da_word = unknown_words
                else:
                    enemy_words, team_words, neutral_words, da_word = unknown_words 

                board_words = team_words, enemy_words, neutral_words, da_word

                CCG.update(board_words)
                
                hintword, hintscore, _ = CCG.give_hints()
                clue = (hintword[0], hintscore)

                UI.console_print("Team " + ("Blue" if TurnTracker.is_blue_turn() else "Red")
                                         + " created the clue " + str(clue[0]) + ", " + str(clue[1]) + ".")
                TurnTracker.increment_turn()
                guess_number = 0

            """ User Input Operations """
            #If the user is trying to quit, exit the program
            if key in ("E"):
                done = True
                continuing = False
            elif key in ("R"):
                done = True
                continuing = True
            elif key in ("S"):
                TurnTracker.increment_turn()
                clue = None
            #If the user has clicked, get their mouse position and perform selection operations
            elif key in ("MOUSE_PRESS"):
                i, j = UI.get_mouse_cell_pos(mouse_real_pos)
                j -= UI.chest_start_pos

                #If a player is selecting a chest during their turn
                if 0 <= i < 5 and 0 <= j < 5 and not CellStates.get_flag_known((i,j)) and TurnTracker.is_guesser_turn() and TurnTracker.is_game_ongoing():
                    guess_number += 1
                    #Set the cell to known
                    word, word_type = CellStates.set_flag_known((i,j))
                    #Get the current unknown words (may have changed or been unchanged)
                    unknown_words = CellStates.get_all_unknown_words()

                    UI.console_print("Team " + ("Blue" if TurnTracker.is_blue_turn() else "Red") + " selected " + word + " for guess #" + str(guess_number))

                    """ Check if game is over, if not increment turn tracker """
                    #If red has no words left to find or blue has chosen the fail word
                    if unknown_words[0] == [] or (unknown_words[3] == [] and TurnTracker.is_blue_turn()):
                        TurnTracker.set_game_finished()
                        UI.console_print("Red has won! Press R to start a new round.")
                    #If blue has no words left to find or red has chosen the fail word
                    elif unknown_words[1] == [] or (unknown_words[3] == [] and TurnTracker.is_red_turn()):
                        TurnTracker.set_game_finished()
                        UI.console_print("Blue has won! Press R to start a new round.")
                    # If the guessing team has exceeded the number of allowed guesses or selected a word not belonging to them
                    elif guess_number > hintscore or word_type != ("R" if TurnTracker.is_red_turn() else "B"):
                        UI.console_print("Team " + ("Blue" if TurnTracker.is_blue_turn() else "Red") + " has ended their turn." )
                        TurnTracker.increment_turn()
                        clue = None


try:
    pg.init()
    UI = UserInterface()
    main(UI)
finally:
    del UI
    pg.quit()

[remove_unseen_words] OUTPUT: lochness not found in fasttext -> excluded from words list
[remove_unseen_words] OUTPUT: scubadiver not found in fasttext -> excluded from words list
appended  cock type <class 'str'>
